In [ ]:
from collections import Counter
import pandas as pd
df_train=pd.read_csv('/home/kesci/input/bytedance/train_final.csv',names=['qId','q','aId','a','target'],nrows=50000000)
df_test=pd.read_csv('/home/kesci/input/bytedance/bytedance_contest.final_2.csv',names=['qId','q','aId','a'])
df_test['a']=df_test['a'].apply(lambda x:x[:-1])

In [ ]:
train_qs = pd.Series(df_train['q'].tolist() + df_train['a'].tolist())
import gc
gc.collect()
words = (" ".join(train_qs)).split()#这里可以改进
counts = Counter(words)
del words
gc.collect()
del train_qs
gc.collect()
train_qs=pd.Series(df_test['q'].tolist())
words = (" ".join(train_qs)).split()#这里可以改进
counts.update(words)
del train_qs,words
gc.collect()
train_qs=pd.Series(df_test['a'].tolist())
words = (" ".join(train_qs)).split()#这里可以改进
counts.update(words)
#内存限制，逐步更新

In [ ]:
print(len(train_qs))
del train_qs,words
gc.collect()
import pickle
def save_obj(obj, name ):
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
save_obj(counts,'dict')

In [ ]:
import pickle
from collections import Counter
import pandas as pd
def load_obj(name ):
    with open( name + '.pkl', 'rb') as f:
        return pickle.load(f)
counts=load_obj('dict')
def get_weight(count, eps=10000, min_count=2):
        return 0 if count < min_count else 1 / (count + eps)
weights = {word: get_weight(count) for word, count in counts.items()}
del words,train_qs
import gc
gc.collect()

In [ ]:
def word_share_fun(df_):
    print('start')
    result=[]
    
    def word_shares(q,a):
        q1_list = q.split()
        q1 = set(q1_list)
        q1words = q1
        if len(q1words) == 0:
            result.append([0,0,0,0,0,0])
        q2_list = a.split()
        q2 = set(q2_list)
        q2words = q2
        if len(q2words) == 0:
            result.append([0,0,0,0,0,0])
    
        words_hamming = sum(1 for i in zip(q1_list, q2_list) if i[0]==i[1])/max(len(q1_list), len(q2_list))
    
        q1_2gram = set([i for i in zip(q1_list, q1_list[1:])])
        q2_2gram = set([i for i in zip(q2_list, q2_list[1:])])
    
        shared_2gram = q1_2gram.intersection(q2_2gram)
    
        shared_words = q1words.intersection(q2words)
        shared_weights = [weights.get(w, 0) for w in shared_words]
        q1_weights = [weights.get(w, 0) for w in q1words]
        q2_weights = [weights.get(w, 0) for w in q2words]
        total_weights = q1_weights + q2_weights
    
        R1 = np.sum(shared_weights) / np.sum(total_weights) #tfidf share
        R2 = len(shared_words) / (len(q1words) + len(q2words) - len(shared_words)) #count share

        Rcosine_denominator = (np.sqrt(np.dot(q1_weights,q1_weights))*np.sqrt(np.dot(q2_weights,q2_weights)))
        Rcosine = np.dot(shared_weights, shared_weights)/Rcosine_denominator
        if len(q1_2gram) + len(q2_2gram) == 0:
            R2gram = 0
        else:
            R2gram = len(shared_2gram) / (len(q1_2gram) + len(q2_2gram))
        result.append([R1, R2, len(shared_words),R2gram, Rcosine, words_hamming])
    df_.apply(lambda row: word_shares(row['q'], row['a']), axis=1)
    return pd.DataFrame(result)

In [ ]:
df1=df_train[['q','a']].iloc[0:10000000,]
df2=df_train[['q','a']].iloc[10000000:20000000,]
df3=df_train[['q','a']].iloc[20000000:30000000,]
df4=df_train[['q','a']].iloc[30000000:40000000,]
df5=df_train[['q','a']].iloc[40000000:50000000,]

df6=df_test[['q','a']].iloc[0:10000000,]
df7=df_test[['q','a']].iloc[10000000:20000000,]
df8=df_test[['q','a']].iloc[20000000:30000000,]
df9=df_test[['q','a']].iloc[30000000:40000000,]
df10=df_test[['q','a']].iloc[40000000:50000000,]
df11=df_test[['q','a']].iloc[50000000:60000000,]
df12=df_test[['q','a']].iloc[60000000:70000000,]
df13=df_test[['q','a']].iloc[70000000:80000000,]
df14=df_test[['q','a']].iloc[80000000:90000000,]
df15=df_test[['q','a']].iloc[90000000:,]

In [ ]:
del df_train,df_test
from multiprocessing import cpu_count
from multiprocessing import Pool
print(cpu_count())
p= Pool(15)
df1=p.apply_async(word_share_fun, args=(df1,))
df2=p.apply_async(word_share_fun, args=(df2,))
df3=p.apply_async(word_share_fun, args=(df3,))
df4=p.apply_async(word_share_fun, args=(df4,))
df5=p.apply_async(word_share_fun, args=(df5,))
df6=p.apply_async(word_share_fun, args=(df6,))
df7=p.apply_async(word_share_fun, args=(df7,))
df8=p.apply_async(word_share_fun, args=(df8,))
df9=p.apply_async(word_share_fun, args=(df9,))
df10=p.apply_async(word_share_fun, args=(df10,))
df11=p.apply_async(word_share_fun, args=(df11,))
df12=p.apply_async(word_share_fun, args=(df12,))
df13=p.apply_async(word_share_fun, args=(df13,))
df14=p.apply_async(word_share_fun, args=(df14,))
df15=p.apply_async(word_share_fun, args=(df15,))
p.close()
p.join()

In [ ]:
df1=df1.get()
df2=df2.get()
df3=df3.get()
df4=df4.get()
df5=df5.get()
df6=df6.get()
df7=df7.get()
df8=df8.get()
df9=df9.get()
df10=df10.get()
df11=df11.get()
df12=df12.get()
df13=df13.get()
df14=df14.get()
df15=df15.get()

In [ ]:
train=pd.concat([df1,df2,df3,df4,df5])
test=pd.concat([df6,df7,df8,df9,df10,df11,df12,df13,df14,df15])
train.to_csv('/home/kesci/work/train_feature_ws_final.csv',index=False)
test.to_csv('/home/kesci/work/test_feature_ws_final.csv',index=False)